# Regularity Feature Generation for A1 and A2

In [ ]:
#import required tools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import timedelta, datetime 
from scipy.spatial.distance import jensenshannon
from scipy import stats

In [2]:
DATA_DIR = '...' # You many change the directory

#import datasets, the form of the dataframe necessary is described in our report.

---

# A1

Below we define all necessary functions to generate the regularity features for our lernnavi data as described in the paper: "How to Quantify Student Regularity?" from the ML4ED lab. They were taken varbatim from the corresponding github repository, with minor adjustments. 

In [ ]:
def similarity_days(wi, wj):
    m1, m2 = np.where(wi == 1)[0], np.where(wj == 1)[0]
    if len(m1) == 0 or len(m2) == 0:
        return 0
    return len(np.intersect1d(m1, m2)) / max(len(m1), len(m2))

In [ ]:
def chi2_divergence(p1, p2, a1, a2):
    a = p1 - p2
    b = p1 + p2
    frac = np.divide(a, b, out=np.zeros(a.shape, dtype=float), where=b != 0)
    m1 = np.where(a1 > 0)[0]
    m2 = np.where(a2 > 0)[0]
    union = set(m1) & set(m2)
    if len(union) == 0: return np.nan
    return 1 - (1 / len(union)) * np.sum(np.square(frac))

In [ ]:
def time_measures(data, mode, weeks):
    """
    Return the PDH and PWD measures up to a given week, weeks
    """
    
    data = data[data['week'] < weeks]
    if mode == 'dayhour': #PDH
        hours = data['date'].dt.hour.astype(int).to_list()
        activity = np.array([hours.count(h) for h in np.arange(24)])
        if np.sum(activity) == 0:
            print("Feature is invalid: the dayhour mode is invalid. Returning nan.")
            return np.nan
        entropy = stats.entropy(activity / np.sum(activity))
        return (np.log(24) - entropy) * np.max(activity)
    elif mode == 'weekday': #PWD
        weekdays = data['date'].dt.weekday.astype(int).to_list()
        activity = np.array([weekdays.count(h) for h in np.arange(7)])
        if np.sum(activity) == 0:
            print("Feature is invalid: the weekday mode is invalid. Returning nan.")
            return np.nan
        entropy = stats.entropy(activity / np.sum(activity))
        return (np.log(7) - entropy) * np.max(activity)

In [ ]:
#necessary to accurately calculate the week of a year given a timestamp
def create_week_number(d):
    return (d.isocalendar()[0] - 2021) * 53 + d.isocalendar()[1] - 1
# https://stackoverflow.com/questions/59425176/how-to-continue-the-week-number-when-the-year-changes-using-pandas

In [ ]:
def profile_similarity(data, mode, weeks):
    """
    Return the WS1, WS2 and WS3 measures up to a given week, weeks
    """
    
    data['weekday'] = data['date'].dt.weekday.astype(int)
    data = data[data['week'] < weeks]
    
    workload = np.zeros((weeks, 7))
    workload[data['week'], data['weekday']] += 1
    hist = workload / np.sum(workload)

    # Hours of activity starting at midnight of the first timestamp
    hours = (data['date']).values.astype(np.int64) // 10 ** 9 // 3600
    min_day = data.date.min() # First day of activity
    # Make the hours start from midnight of the first day
    hours -= int(datetime(min_day.year, min_day.month, min_day.day).timestamp() / 3600)

    period_length = weeks * 7 * 24
    activity = np.array([int(t in hours) for t in range(period_length)]).reshape((weeks, 7 * 24))
    activity = np.array([week.reshape((7, 24)).sum(axis=1) for week in activity])  # shape (weeks, 7)
    if mode == 'm1': #WS1
        return np.mean([similarity_days(workload[i], workload[j]) for i in range(workload.shape[0]) for j in range(i+1, workload.shape[0])])
    elif mode == 'm2': #WS2
        res = []
        for i in range(activity.shape[0]):
            for j in range(i + 1, activity.shape[0]):
                if not activity[i].any() or not activity[j].any():
                    continue
                res.append(1 - jensenshannon(activity[i], activity[j], 2.0))
        if len(res) == 0:
            #print("Feature is invalid. Will return nan")
            return np.nan
        return np.mean(np.clip(np.nan_to_num(res), 0, 1))
    elif mode == 'm3': #WS3
        res = []
        for i in range(activity.shape[0]):
            for j in range(i + 1, activity.shape[0]):
                if not activity[i].any() or not activity[j].any():
                    continue
                res.append(chi2_divergence(activity[i], activity[j], hist[i], hist[j]))
        if len(res) == 0:
            return np.nan
        return np.mean(np.nan_to_num(res))

In [ ]:
def fourier_transform(Xi, f, n):
    return np.dot(np.exp(-2j * np.pi * f * n), Xi)

In [1]:
def freq(data, mode, weeks):
    """
    Return the FDH, FWD, and FWH measures up to a given week, weeks 
    """
    
    if mode == 'm1': #FDH
        # Convert date to hours starting from 0
        hours = data['date'].values.astype(np.int64) // 10 ** 9 // 3600
        hours -= min(hours)
        period_length = weeks * 7 * 24
        activity = np.array([int(t in hours) for t in range(period_length)])  # 1 if active at hour t 0 o.w.
        if np.sum(activity) == 0:
            #logging.debug('feature {} is invalid: the m1 mode is invalid'.format(self.name))
            return np.nan #Feature.INVALID_VALUE
        n = np.arange(period_length)
        return abs(fourier_transform(activity, 1 / 24, n))

    elif mode == 'm2': #FWH
        period_length = weeks * 7 * 24
        hours = data['date'].values.astype(np.int64) // 10 ** 9 // 3600
        hours -= min(hours)
        activity = np.array([int(t in hours) for t in range(period_length)])
        n = np.arange(period_length)
        return abs(fourier_transform(activity.flatten(), 1 / (7 * 24), n))

    elif mode == 'm3': #FWD
        # Convert date to days starting from 0
        days = data['date'].values.astype(np.int64) // 10 ** 9 // (24 * 3600)
        days -= min(days)
        period_length = weeks * 7
        activity = np.array([int(d in days) for d in range(period_length)])  # 1 if active at day d 0 o.w.
        n = np.arange(period_length)
        return abs(fourier_transform(activity, 1 / 7, n))

In [ ]:
def compute_agg_regularity(data, weeks):
    """
    Return regularity measures for activity aggregated over a predefined number of weeks
    """
    
    #time based measures
    PDH = data.groupby('user_id').apply(lambda x: time_measures(x, 'dayhour', weeks)).reset_index()
    PWD = data.groupby('user_id').apply(lambda x: time_measures(x, 'weekday', weeks)).reset_index(drop=True)
    
    #profile similarity 
    WS1 = data.groupby('user_id').apply(lambda x: profile_similarity(x, 'm1', weeks)).reset_index(drop=True)
    WS2 = data.groupby('user_id').apply(lambda x: profile_similarity(x, 'm2', weeks)).reset_index(drop=True)
    WS3 = data.groupby('user_id').apply(lambda x: profile_similarity(x, 'm3', weeks)).reset_index(drop=True)
    
    #frequency based measures
    FDH = data.groupby('user_id').apply(lambda x: freq(x, 'm1', weeks)).reset_index(drop=True)
    FWH = data.groupby('user_id').apply(lambda x: freq(x, 'm2', weeks)).reset_index(drop=True)
    FWD = data.groupby('user_id').apply(lambda x: freq(x, 'm3', weeks)).reset_index(drop=True)
    
    reg_features = pd.concat([PDH,PWD,WS1,WS2,WS3,FDH,FWH,FWD], axis=1, ignore_index=False)
    
    return reg_features 

---

## A2

Below we define the function necessary to compute the behavioural features used in approach A2, as described in our report. 

In [3]:
def behavioural_features(df, lvl_chks):
    """
    Compute features to feed decision tree classifier for level check scores.
    
    Input: df - complete DataFrame with user activity data as described in report section 3
           lvl_chks - DataFrame where first column is user_id and second is week on which user_id does a level check
    Output: behavioural as described above features for each row of lvl_chks
    """
    
    users = list(set(lvl_chks.user_id))
    behavioural_features = pd.DataFrame(columns = ['num_actions_per_week',
                                                   'eng_score',
                                                   'FDH',
                                                   'FWH',
                                                   'FWD',
                                                   'num_weeks_on'],
                                        index = [lvl_chks['user_id'], lvl_chks['week']]) #same nrows lvl_chks
    
    behavioural_features_dicts = []
    
    for user in users:
        lvl_chk_weeks = lvl_chks[lvl_chks['user_id'] == user].week.tolist()
        
        for j, week in enumerate(lvl_chk_weeks): 
            df_user_week = df[(df['user_id'] == user) & (df['week'] <= week)]
            
            if df_user_week.empty:
                behavioural_features_dicts.append({'num_actions_per_week' : 0,
                                                   'eng_score' : 0,
                                                   'FDH' : 0,
                                                   'FWH' : 0,
                                                   'FWD' : 0,
                                                   'num_weeks_on' : 0,
                                                   'user_id' : user,
                                                   'week' : week,
                                                   'validated' : lvl_chks[(lvl_chks['user_id'] == user) &
                                                                          (lvl_chks['week'] == week)]['validated'].values[0]})
                                                
                continue 
            
            sum_actions_week = df_user_week['action'].count() 
            sum_eng_actions_week = df_user_week[(df_user_week['action'] == 'SUBMIT_ANSWER') |
                                                (df_user_week['action'] == 'GO_TO_THEORY')]['action'].count()
            num_actions_per_week = sum_actions_week / (j+1)
            eng_prop = sum_eng_actions_week / sum_actions_week
            eng_score = 1 if eng_prop >= 0.25 else 0 
            
            FDH = freq(df_user_week, 'm1', j+1) 
            FWH = freq(df_user_week, 'm2', j+1)
            FWD = freq(df_user_week, 'm3', j+1)
            
            num_weeks_on = j+1 #(i+1) counts how many weeks for which we have observed the user use the platform
            
            behavioural_features_dicts.append({'num_actions_per_week' :num_actions_per_week,
                                               'eng_score' : eng_score,
                                               'FDH' : FDH,
                                               'FWH' : FWH,
                                               'FWD' : FWD,
                                               'num_weeks_on' : num_weeks_on,
                                               'user_id' : user,
                                               'week' : week,
                                               'validated' : lvl_chks[(lvl_chks['user_id'] == user) &
                                                                      (lvl_chks['week'] == week)]['validated'].values[0]})
            
    behavioural_features = pd.DataFrame(behavioural_features_dicts)
    #print(behavioural_features)
        
    return behavioural_features 

---

# Appendix: Producing PWD/PWH/PDH plots

In [4]:
def time_measures_plot_student(student, data, mode, weeks):
    """
    Produce plots consistent with PDH and PWD measures as described in paper from ML4ED lab. 
    
    Input: student - user_id of a given user
           data - complete, clean dataframe of all user activity data
           mode - identifier of PDH or PWD
           weeks - number of weeks over which the compute the features
    
    Output: corresponding plots
    """
    data = data[data['user_id'] == student]
    data = data[data['week'] < weeks]
    if mode == 'dayhour':
        hours = data['date'].dt.hour.astype(int).to_list()
        activity = np.array([hours.count(h) for h in np.arange(24)])
        time_measure_data = pd.DataFrame({'hour' : np.arange(24), 'count' : activity})
        
        ax = time_measure_data.plot.bar(x='hour', y='count', color=greyscale[1])
        ax.set_xticklabels(['0','1','2','3','4','5','6'],rotation=0)
        ax.set_xlabel("Hour of day")
        ax.set_ylabel("Count")
        ax.set_title("Hours of activity for user {}".format(int(student)))
        
        ax.get_legend().remove()
        ax.get_figure().savefig(FIG_DIR + 'PDHex.jpg')

    elif mode == 'weekday':
        weekdays = data['date'].dt.weekday.astype(int).to_list()
        activity = np.array([weekdays.count(h) for h in np.arange(7)])
        time_measure_data = pd.DataFrame({'weekday' : np.arange(7), 'count' : activity})
        
        ax = time_measure_data.plot.bar(x='weekday', y='count', color=greyscale[1])
        ax.set_xticklabels(['0','1','2','3','4','5','6'],rotation=0)
        ax.set_xlabel("Day of week")
        ax.set_ylabel("Count")
        ax.set_title("Days of activity for user {}".format(int(student)))
        
        ax.get_legend().remove()
        ax.get_figure().savefig(FIG_DIR + 'PWHex.jpg')